Based on [Sentiment Analysis on Reddit News tutorial](http://www.learndatasci.com/sentiment-analysis-reddit-headlines-pythons-nltk/)

Categorize news articles as positive, negative or neutral based on overall sentiment of content

In [50]:
import requests
import json
import time
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

### Collect articles from Reddit politics channel

In [46]:
header = {'User-Agent': 'windows:r/politics.single.result:v1.0' +
       '(by /u/anaerobeth)'}
url = 'https://www.reddit.com/r/politics/.json'
request = requests.get(url, headers=header)
data = json.loads(request.text)

In [ ]:
# collect 1000 articles in batches of 250
posts = data['data']['children']
number_of_posts = 0

while len(posts) <= 100:
    time.sleep(2)
    last = posts[-1]['data']['name']
    url = 'https://www.reddit.com/r/politics.json?after=' + str(last)
    request = requests.get(url, headers=header)
    data = json.loads(request.text)
    posts += data['data']['children']
    if number_of_posts == len(posts):
        break
    else:
        number_of_posts = len(posts)

### Run sentiment analysis

In [ ]:
 sia = SIA()
positives = []
negatives = []

for post in posts:
    response = sia.polarity_scores(post['data']['title'])
    if response['compound'] > 0.2:
        positives.append(post['data']['title'])
    elif response['compound'] < -0.2:
        negatives.append(post['data']['title'])

with open("positive_news_titles.txt", "w", encoding='utf-8', errors='ignore') as f_pos:
    for post in positives:
        f_pos.write(post + "\n")

with open("negative_news_titles.txt", "w", encoding='utf-8', errors='ignore') as f_neg:
    for post in negatives:
        f.neg.write(post + "\n")
    